In [ ]:
import pandas as pd
import numpy as np 
import glob
from tqdm import tqdm
import sys
import os
from pathlib import Path

BASE_DIR = Path.cwd()
TARGET_DIR = BASE_DIR / "data" / "raw" / "high_10m" / "*.csv.gz"
files = sorted(glob.glob(str(TARGET_DIR)))

In [ ]:
days = []

for fn in tqdm(files, desc= "Reading files"):
    day = pd.read_csv(fn, compression= "gzip", parse_dates= ["DATE"])
    days.append(day)

df = pd.concat(days, ignore_index= True)

In [ ]:
print("Shape of the data: ", df.shape)
print("Number of stocks: ", df["SYMBOL"].nunique())


In [ ]:
df = df.sort_values(by=["SYMBOL", "DATE"])
df.isnull().sum()

### IF WE WANT TO TRADE DAILY AND NOT CARE ABOUT THE OVERNIGHT SHIFT, WE CALCULATE THE RETURN ON THE DAILY BASIS, MEANING I GROUPBY THE DATE AND WE START WITH A RETURN OF 0 EVERY DAY, WE CAN CHANGE THIS IN THE FUTURE

In [ ]:
# Calculate the return on the column MID_OPEN, for each stock "SYMBOL" and on each day "DATE"
df["RETURN_NoOVERNIGHT"] = (df.groupby(["SYMBOL", "DATE"])["MID_OPEN"].pct_change()) # Best way to calcuate the return
df["RETURN_NoOVERNIGHT"] = df["RETURN_NoOVERNIGHT"].fillna(0)

df["LOG_RETURN_NoOVERNIGHT"] = np.log(1 + df["RETURN_NoOVERNIGHT"])
df["LOG_RETURN_NoOVERNIGHT"] = df["LOG_RETURN_NoOVERNIGHT"].fillna(0)

In [ ]:
# Calculate the return on the column MID_OPEN, for each stock "SYMBOL" 
df["RETURN_SiOVERNIGHT"] = (df.groupby("SYMBOL")["MID_OPEN"].pct_change()) # Best way to calcuate the return
df["RETURN_SiOVERNIGHT"] = df["RETURN_SiOVERNIGHT"].fillna(0)

df["LOG_RETURN_SiOVERNIGHT"] = np.log(1 + df["RETURN_SiOVERNIGHT"])
df["LOG_RETURN_SiOVERNIGHT"] = df["LOG_RETURN_SiOVERNIGHT"].fillna(0)

In [ ]:
# SAVE THE PARQUET IN THE DATA FOLDER FOR EASY ACCESS NEXT TIME
SAVE_DIR = BASE_DIR / "data" / "processed" / "high_10m.parquet"
df.to_parquet(SAVE_DIR, index = False)

In [ ]:
print("Number of intraday returns per stock: ", (df.groupby(['SYMBOL', 'DATE'])['TIME'].nunique()).nunique())
print(df.isnull().sum())

In [ ]:
df.drop(['ALL_EX', 'SUM_DELTA'], axis=1, inplace=True)

print("DataFrame Info:")
df.info()
print("-" * 50)


In [ ]:
print(df['SYMBOL'].nunique()) # How many unique symbols are there?
print(df['SYMBOL'].value_counts()) # Top 5 most frequent symbols
# print(df[df['SYMBOL'] == 'AAA']) # Look at the first few rows for a specific symbol
# If there are few unique values, consider converting to 'category' dtype to save memory:
# df['SYMBOL'] = df['SYMBOL'].astype('category')

print(df['TIME'].nunique())
print(df['TIME'].value_counts()) # Look at the first few time entries

# If few unique values, convert to 'category':
# df['ALL_EX'] = df['ALL_EX'].astype('category')